In [1]:
import os
import sys
import opendatasets as od
from joblib import parallel_backend
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.optimize as sopt
import scipy.stats as sstats
import seaborn as sns
import sklearn.ensemble
import sklearn.tree
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from matplotlib import animation, pyplot, rc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.utils import resample
import gc
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from datetime import datetime, time
import json 

PLOT_MODULE = "/home/meks/Desktop/Flight-Delay-Prediction-Project/ml/"
sys.path.append(PLOT_MODULE)

from EDA_plottings import run_plots

# garbage collector so we can save memory
import gc

# Problem Statement and Objectives


# Data Colection


## Importing preprocessed data 


We will be importing data that was merged data of flight dataset from kaggle
and weather from visual crossing (https://www.visualcrossing.com) page. Thanks for cooperation with them,
we were allowed to make large queries about weather in certain places and time.

In [2]:
DATASET_PATH = "/home/meks/Desktop/data_cale.csv"


Lets open it:

In [3]:
pd.set_option('display.max_columns', None)

#NOT SURE ABOUT CRS DEST DISTANCE
dataset = pd.read_csv(DATASET_PATH, dtype={'snow':'uint16', 'rain':'uint16','freezingrain':'uint16','ice':'uint16',
                                                 'FL_MONTH':'uint16', 'FL_DAY' : 'uint16', 'FL_YEAR':'uint32', 'OP_CARRIER':'uint16',
                                           'DELAY' : 'uint16', 'DEST' : 'uint16', 'CRS_ARR_TIME' : 'uint16', 'DISTANCE' : 'uint16'})



In [ ]:
dataset.head(n=100)

In [ ]:
dataset.columns.values

# A small peek on size of data




It's definetely too much, we need to make it smaller, especially the string columns. We would like to convert big string data into mappings of ints to strings. Lets look at example of big columns:

In [ ]:
## do wyjebania imo  AIRLINE_DOT,AIRLINE_DOT,DOT_CODE,ORIGIN,DEST ewentualnie zostawic skroty i zrobic jakis slownik do wypisywania
for col_name in dataset.columns.values:
  print(f"Size in mb of {col_name} : {sys.getsizeof(dataset[col_name]) / 10 ** 6} mb, Type {dataset[col_name].dtype}")
print(f"Data size : {sys.getsizeof(dataset) / 10 ** 9} in GB")

# Data Discovery

Info about columns:

In [ ]:
print(f"There ara {dataset.shape[0]} rows and {dataset.shape[1]} columns.")

In [ ]:
dataset.columns.values

In [ ]:
dataset.info()

Mean of delays:

In [ ]:
delays = dataset['DELAY']
delays.mean()

Check how many nans in dataframe:

Maybe we should remove windgust? So many NaNs, probably no colleration, ID?

In [ ]:

def check_nans():
  # since all len's are same
  n = len(dataset['DELAY'])
  for col in dataset.columns.values:
    nans = dataset[col].isna().sum()
    print(f"Col name: {col} nans: {nans}   {(nans / n) * 100} %")
  print(f"There are {n} rows")

check_nans()


# Time for some plots:

Pie graph of delays:

In [ ]:
def wykreskolowydelay(df):
    delayed_flights = df[df['DELAY'] > 0]
    percentages = [len(delayed_flights) / len(df) * 100, 100 - len(delayed_flights) / len(df) * 100]
    labels = ['Delayed', 'Non-delayed']
    plt.figure(figsize=(8, 8))
    plt.pie(percentages, labels=labels, autopct='%1.1f%%', colors=['skyblue', 'lightgreen'], startangle=90)
    plt.title('Percentage of delayed')
    plt.show()

def wykresmiesiacdealay(df):
    monthly_delays = df.groupby(df['FL_MONTH'])['DELAY'].mean() * 100
    plt.figure(figsize=(10, 6))
    monthly_delays.plot(kind='bar', color='skyblue', edgecolor='black')
    plt.title('Procentowe opóźnienia')
    plt.xlabel('Miesiąc')
    plt.ylabel('Procentowe opóźnienie')
    plt.xticks(rotation=0)
    plt.show()

def wykresmiesiacliczbalotow(df):
    monthly_delays = df.groupby(df['FL_MONTH'])['id'].count()
    print(monthly_delays)
    plt.figure(figsize=(10, 6))
    monthly_delays.plot(kind='bar', color='skyblue', edgecolor='black')
    plt.title('Liczba lotów na miesiąc')
    plt.xlabel('Miesiąc')
    plt.ylabel('Liczba lotów')
    plt.xticks(rotation=0)
    plt.show()
def wykresmiesiacopoznienie(df):
    monthly_delays = df.groupby('FL_MONTH')['DELAY'].sum()
    plt.figure(figsize=(10, 6))
    print(monthly_delays)
    monthly_delays.plot(kind='bar', color='skyblue', edgecolor='black')
    plt.title('Ilość opóźnionych lotów')
    plt.xlabel('Miesiąc')
    plt.ylabel('Liczba opóźnień')
    plt.xticks(rotation=0)
    plt.show()

graphs = [wykreskolowydelay, wykresmiesiacdealay, wykresmiesiacliczbalotow, wykresmiesiacopoznienie]

for g in graphs:
    g(dataset)

# Data Preprocessing

## Data Cleaning

Let's remove windgust, since it doesn't provide any important data (does it?) and contains of lots of nans.
Also, 8 percents of snow might be too much. For now, we will remove it and maybe later on we will return it back.

In [ ]:
dataset = dataset.drop(['WINDGUST', 'SNOW', "SNOWDEPTH", "ID", "id"], axis = 1, errors = 'ignore')
dataset.head(100)

In [ ]:
check_nans()

Now it would be great if we removed NaN rows, I will remove all NaN rows from MOONPHASE, CONDITIONS etc. I know only by seeing those rows, that there since moonphase and conditions and cloudcover etc. have 0.002481972697193719 % of nans, they have nans on all rows. So i need to remove only moonphase visibility and pressure rows with NaN's.

In [ ]:
nan_cols = ["MOONPHASE", 'VISIBILITY', 'PRESSURE']

dataset.dropna(subset = nan_cols, inplace = True)

Now lets look at nans in overall:

In [ ]:
check_nans()

We have our data cleaned. Data we imported had ~12.65 mln. rows, after cleaning we have ~12.64 mln. rows. (???)

## Custom Transformers

## Feature Scaling

## Tranformation Pipelines

# Exploratory Data Analysis

# Machine Learning Proccess

In [ ]:
def LogREG(X_train,X_test,y_train,y_test):
    model = LogisticRegression(max_iter=1000)

    # Different logistic regression parameters to try
    param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'solver': ['liblinear', 'newton-cg', 'lbfgs', 'sag']}

    # Use GridSearchCV for hyperparameter tuning
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    # Get the best parameters and best model from the grid search
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    # Make predictions on the test set
    y_pred = best_model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)

    # Print results
    print("LR best parameters:", best_params)
    print("Accuracy:", accuracy)

def SVM(X_train, X_test, y_train, y_test):
    clf = svm.SVC()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)

    print(f'Dokładność klasyfikacji: {accuracy:.2f}')

    # Wyświetlanie pełnego raportu klasyfikacji
    print('\nRaport klasyfikacji:')
    print(classification_report(y_test, y_pred))


def DecisionTree(X_train, X_test, y_train, y_test, max_depth=4):
    # Initializing the decision tree classifier
    clf = DecisionTreeClassifier(max_depth=max_depth, random_state=42, criterion='gini')

    # Training the classifier on the training data
    clf.fit(X_train, y_train)

    # Predicting labels for the test data
    y_pred = clf.predict(X_test)

    # Evaluating the classifier's accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Classification Accuracy: {accuracy:.2f}')

    # Displaying the full classification report
    print('\nClassification Report:')
    print(classification_report(y_test, y_pred))

    # Plotting the decision tree
    plt.figure(figsize=(12, 8))
    plot_tree(clf, filled=True, feature_names=X_train.columns, class_names=['Class 0', 'Class 1'])
    plt.show()

In [ ]:
X = dataset.loc[:, dataset.columns != "DELAY"]
y = dataset["DELAY"]

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.1)





In [ ]:
#LogREG(X_train, X_test, y_train, y_test)
#SVM(X_train, X_test, y_train, y_test)
DecisionTree(X_train, X_test, y_train, y_test)